In [23]:
import pandas as pd
import plotly.graph_objects as go
import oanda_utils

In [24]:
plot_cols = ['ENTRY', 'STOPLOSS', 'TAKEPROFIT']
plot_colors = ['#043ef9', '#eb5334', '#34eb37']
def plot_candles(df_plot):
    fig = go.Figure()
    fig.add_trace(go.Candlestick(
        x = df_plot.time, open=df_plot.mid_o, high=df_plot.mid_h, low=df_plot.mid_l, close=df_plot.mid_c,
        line=dict(width=1), opacity=1,
        increasing_fillcolor='#24A06B',
        decreasing_fillcolor='#CC2E3C',
        increasing_line_color='#2EC886',
        decreasing_line_color='#FF3A4C'
    ))
    for i in range(0, 3):
        fig.add_trace(go.Scatter(
            x=df_buys.time,
            y=df_buys[plot_cols[i]],
            mode='markers',     # this hides the line that connects the dots
            marker=dict(color=plot_colors[i], size=12)
        ))
    for i in range(0, 3):
        fig.add_trace(go.Scatter(
            x=df_sells.time,
            y=df_sells[plot_cols[i]],
            mode='markers',     # this hides the line that connects the dots
            marker=dict(color=plot_colors[i], size=12)
        ))
    fig.update_layout(width=1000,height=400,
        margin=dict(l=10, r=10, b=10, t=10),
        font=dict(size=10,color="#e1e1e1"),
        paper_bgcolor="#1e1e1e", 
        plot_bgcolor="#1e1e1e")

    fig.update_xaxes(
        gridcolor="#1f292f",
        showgrid=True,fixedrange=True,rangeslider=dict(visible=False),
        rangebreaks=[
            dict(bounds=["sat", "mon"])
        ]
    )

    fig.update_yaxes(
        gridcolor="#1f292f",
        showgrid=True)

    fig.show()  

In [25]:
pair = "USD_JPY"
granularity = "H4"
df_raw = pd.read_pickle(oanda_utils.get_his_data_filename(pair,granularity))

In [26]:
non_cols = ['time', 'volume']
mod_cols = [x for x in df_raw.columns if x not in non_cols]
df_raw[mod_cols] = df_raw[mod_cols].apply(pd.to_numeric)

In [27]:
df_raw.head()

,time,volume,mid_o,mid_h,mid_l,mid_c,bid_o,bid_h,bid_l,bid_c,ask_o,ask_h,ask_l,ask_c
0,2018-01-01 22:00:00+00:00,2968,112.666,112.793,112.582,112.723,112.626,112.786,112.559,112.716,112.706,112.800,112.601,112.730
3,2018-01-02 02:00:00+00:00,1009,112.720,112.751,112.666,112.688,112.712,112.744,112.657,112.683,112.728,112.758,112.674,112.694
6,2018-01-02 06:00:00+00:00,2226,112.691,112.700,112.249,112.252,112.684,112.694,112.242,112.245,112.698,112.705,112.256,112.258
9,2018-01-02 10:00:00+00:00,2056,112.254,112.282,112.086,112.124,112.247,112.275,112.079,112.119,112.260,112.288,112.092,112.130
12,2018-01-02 14:00:00+00:00,3781,112.122,112.304,112.058,112.285,112.114,112.297,112.051,112.278,112.129,112.311,112.065,112.292


In [28]:
df_raw.columns

Index(['time', 'volume', 'mid_o', 'mid_h', 'mid_l', 'mid_c', 'bid_o', 'bid_h',
       'bid_l', 'bid_c', 'ask_o', 'ask_h', 'ask_l', 'ask_c'],
      dtype='object')

In [29]:
SLOSS = 0.4
TPROFIT = 0.8
ENTRY_PRC = 0.1

def direction(row):
    if row.mid_c > row.mid_o:
        return 1
    return -1

def get_signal(row):
    if row.mid_h_prev > row.mid_h and row.mid_l_prev < row.mid_l:
        return row.DIRECTION_prev
    return 0

def get_entry_stop(row):
    if row.SIGNAL == 1:
        return (row.RANGE_prev * ENTRY_PRC) + row.ask_h_prev
    elif row.SIGNAL == -1:
        return row.bid_l_prev - (row.RANGE_prev * ENTRY_PRC)
    else:
        return 0

def get_stop_loss(row):
    if row.SIGNAL == 1:
        return row.ENTRY - (row.RANGE_prev * SLOSS)
    elif row.SIGNAL == -1:
        return row.ENTRY + (row.RANGE_prev * SLOSS)
    else:
        return 0

def get_take_profit(row):
    if row.SIGNAL == 1:
        return row.ENTRY + (row.RANGE_prev * TPROFIT)
    elif row.SIGNAL == -1:
        return row.ENTRY - (row.RANGE_prev * TPROFIT)
    else:
        return 0


In [30]:
df = df_raw.copy()
df['RANGE'] = df.mid_h - df.mid_l
df['mid_h_prev'] = df.mid_h.shift(1)
df['mid_l_prev'] = df.mid_l.shift(1)
df['ask_h_prev'] = df.ask_h.shift(1)
df['bid_l_prev'] = df.bid_l.shift(1)
df['RANGE_prev'] = df.RANGE.shift(1)
df['DIRECTION'] = df.apply(direction, axis=1)
df['DIRECTION_prev'] = df.DIRECTION.shift(1).fillna(0).astype(int)
df.dropna(inplace=True)
df['SIGNAL'] = df.apply(get_signal, axis=1)
df.reset_index(drop=True, inplace=True)

In [31]:
df['ENTRY'] = df.apply(get_entry_stop, axis=1)
df['STOPLOSS'] = df.apply(get_stop_loss, axis=1)
df['TAKEPROFIT'] = df.apply(get_take_profit, axis=1)

In [32]:
df.head()

,time,volume,mid_o,mid_h,mid_l,mid_c,bid_o,bid_h,bid_l,bid_c,...,mid_l_prev,ask_h_prev,bid_l_prev,RANGE_prev,DIRECTION,DIRECTION_prev,SIGNAL,ENTRY,STOPLOSS,TAKEPROFIT
0,2018-01-02 02:00:00+00:00,1009,112.720,112.751,112.666,112.688,112.712,112.744,112.657,112.683,...,112.582,112.800,112.559,0.211,-1,1,1,112.8211,112.7367,112.9899
1,2018-01-02 06:00:00+00:00,2226,112.691,112.700,112.249,112.252,112.684,112.694,112.242,112.245,...,112.666,112.758,112.657,0.085,-1,-1,0,0.0000,0.0000,0.0000
2,2018-01-02 10:00:00+00:00,2056,112.254,112.282,112.086,112.124,112.247,112.275,112.079,112.119,...,112.249,112.705,112.242,0.451,-1,-1,0,0.0000,0.0000,0.0000
3,2018-01-02 14:00:00+00:00,3781,112.122,112.304,112.058,112.285,112.114,112.297,112.051,112.278,...,112.086,112.288,112.079,0.196,1,-1,0,0.0000,0.0000,0.0000
4,2018-01-02 18:00:00+00:00,1268,112.288,112.354,112.244,112.291,112.280,112.347,112.238,112.262,...,112.058,112.311,112.051,0.246,1,1,0,0.0000,0.0000,0.0000


In [33]:
df[df.SIGNAL!=0].shape

(830, 26)

In [34]:
df[df.SIGNAL!=0].head()

,time,volume,mid_o,mid_h,mid_l,mid_c,bid_o,bid_h,bid_l,bid_c,...,mid_l_prev,ask_h_prev,bid_l_prev,RANGE_prev,DIRECTION,DIRECTION_prev,SIGNAL,ENTRY,STOPLOSS,TAKEPROFIT
0,2018-01-02 02:00:00+00:00,1009,112.720,112.751,112.666,112.688,112.712,112.744,112.657,112.683,...,112.582,112.800,112.559,0.211,-1,1,1,112.8211,112.7367,112.9899
12,2018-01-04 02:00:00+00:00,1560,112.731,112.731,112.618,112.661,112.724,112.724,112.611,112.654,...,112.480,112.785,112.455,0.298,-1,1,1,112.8148,112.6956,113.0532
17,2018-01-04 22:00:00+00:00,1563,112.750,112.806,112.730,112.774,112.722,112.799,112.708,112.767,...,112.713,112.846,112.706,0.125,1,-1,-1,112.6935,112.7435,112.5935
21,2018-01-05 14:00:00+00:00,3844,113.213,113.276,113.082,113.160,113.206,113.269,113.076,113.153,...,113.021,113.330,112.999,0.291,-1,-1,-1,112.9699,113.0863,112.7371
27,2018-01-08 14:00:00+00:00,2537,112.928,113.125,112.904,113.082,112.921,113.119,112.898,113.076,...,112.884,113.173,112.877,0.283,1,-1,-1,112.8487,112.9619,112.6223


In [35]:
df.shape

(4685, 26)

How to simulate the 4 candle strategy

In [36]:
class Trade ():
    def __init__(self, row):
        self.candle_date = row.time
        self.direction = row.SIGNAL
        self.entry = row.ENTRY
        self.TP = row.TAKEPROFIT
        self.SL = row.STOPLOSS
        self.running = False
        self.result = None
        self.stopped = None

    def update(self, row):
        if self.running == True:
            self.update_result(row)
        else:
            self.check_entry(row)

    def check_entry(self, row):
        if self.direction == 1 and row.mid_c >= self.entry or self.direction == -1 and row.mid_c <= self.entry:
            self.index = row.name
            self.opened = row.time
            self.running = True

    def update_result(self, row):
        if self.direction == 1:
            if row.mid_c >= self.TP:
                self.result = 2.0
            elif row.mid_c <= self.SL:
                self.result = -1.0
        else:
            if row.mid_c <= self.TP:
                self.result = 2.0
            elif row.mid_c >= self.SL:
                self.result = -1.0

        if self.result is not None:
            self.running = False
            self.stopped = row.time

In [37]:
open_trades = []
closed_trades = []

for index, row in df.iterrows():
    for ot in open_trades:
        ot.update(row)
        if ot.stopped is not None:
            closed_trades.append(ot)
    
    open_trades = [x for x in open_trades if x.stopped is None]

    if row.SIGNAL != 0:
        open_trades = [x for x in open_trades if x.running == True]
        open_trades.append(Trade(row))

In [38]:
len(closed_trades)

297

In [39]:
df_trades = pd.DataFrame.from_dict([vars(x) for x in closed_trades])
df_trades.head()

,candle_date,direction,entry,TP,SL,running,result,stopped,index,opened
0,2018-01-04 02:00:00+00:00,1,112.8148,113.0532,112.6956,False,2.0,2018-01-05 06:00:00+00:00,14,2018-01-04 10:00:00+00:00
1,2018-01-05 14:00:00+00:00,-1,112.9699,112.7371,113.0863,False,-1.0,2018-01-08 18:00:00+00:00,26,2018-01-08 10:00:00+00:00
2,2018-01-08 14:00:00+00:00,-1,112.8487,112.6223,112.9619,False,2.0,2018-01-09 14:00:00+00:00,29,2018-01-08 22:00:00+00:00
3,2018-01-09 18:00:00+00:00,-1,112.3189,111.9901,112.4833,False,2.0,2018-01-10 06:00:00+00:00,35,2018-01-09 22:00:00+00:00
4,2018-01-10 02:00:00+00:00,-1,112.0993,111.6057,112.3461,False,2.0,2018-01-10 10:00:00+00:00,37,2018-01-10 06:00:00+00:00


In [40]:
df[df.SIGNAL!=0].head()

,time,volume,mid_o,mid_h,mid_l,mid_c,bid_o,bid_h,bid_l,bid_c,...,mid_l_prev,ask_h_prev,bid_l_prev,RANGE_prev,DIRECTION,DIRECTION_prev,SIGNAL,ENTRY,STOPLOSS,TAKEPROFIT
0,2018-01-02 02:00:00+00:00,1009,112.720,112.751,112.666,112.688,112.712,112.744,112.657,112.683,...,112.582,112.800,112.559,0.211,-1,1,1,112.8211,112.7367,112.9899
12,2018-01-04 02:00:00+00:00,1560,112.731,112.731,112.618,112.661,112.724,112.724,112.611,112.654,...,112.480,112.785,112.455,0.298,-1,1,1,112.8148,112.6956,113.0532
17,2018-01-04 22:00:00+00:00,1563,112.750,112.806,112.730,112.774,112.722,112.799,112.708,112.767,...,112.713,112.846,112.706,0.125,1,-1,-1,112.6935,112.7435,112.5935
21,2018-01-05 14:00:00+00:00,3844,113.213,113.276,113.082,113.160,113.206,113.269,113.076,113.153,...,113.021,113.330,112.999,0.291,-1,-1,-1,112.9699,113.0863,112.7371
27,2018-01-08 14:00:00+00:00,2537,112.928,113.125,112.904,113.082,112.921,113.119,112.898,113.076,...,112.884,113.173,112.877,0.283,1,-1,-1,112.8487,112.9619,112.6223


In [41]:
df[df.SIGNAL!=0].to_pickle("USD_JPY_H4_trades.pkl")

In [42]:
df_plot = df.iloc[0:60]
df_buys = df_plot[df_plot.SIGNAL == 1]
df_sells = df_plot[df_plot.SIGNAL == -1]

df_trades.head()

,candle_date,direction,entry,TP,SL,running,result,stopped,index,opened
0,2018-01-04 02:00:00+00:00,1,112.8148,113.0532,112.6956,False,2.0,2018-01-05 06:00:00+00:00,14,2018-01-04 10:00:00+00:00
1,2018-01-05 14:00:00+00:00,-1,112.9699,112.7371,113.0863,False,-1.0,2018-01-08 18:00:00+00:00,26,2018-01-08 10:00:00+00:00
2,2018-01-08 14:00:00+00:00,-1,112.8487,112.6223,112.9619,False,2.0,2018-01-09 14:00:00+00:00,29,2018-01-08 22:00:00+00:00
3,2018-01-09 18:00:00+00:00,-1,112.3189,111.9901,112.4833,False,2.0,2018-01-10 06:00:00+00:00,35,2018-01-09 22:00:00+00:00
4,2018-01-10 02:00:00+00:00,-1,112.0993,111.6057,112.3461,False,2.0,2018-01-10 10:00:00+00:00,37,2018-01-10 06:00:00+00:00


In [43]:
df_trades.result.sum()

192.0

In [44]:
plot_candles(df_plot)